<a href="https://colab.research.google.com/github/botkin92/Recommender_systems/blob/lesson_2/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 5.0 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3423256 sha256=b85484084a6cddeed196f13a0a57f31b4e89c7851bcbed9b173579326218d01d
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
# data = pd.read_csv('data/retail_train.csv')
# data.head(2)

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [6]:
data = pd.read_csv('MyDrive/Colab Notebooks/Recommender_systems/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights['item_id'])
    weight = np.array(items_weights['weight'])
            
    return np.random.choice(items, size=5, replace=False, p=weight).tolist()

In [9]:
%%time

# your_code
# Подготовим массив items_weights = popularity
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()

# При расчете весов введем константу 1, чтобы уйти от нулевых значений
sum_log = np.log(popularity['sales_value'].values + 1).sum()
popularity['weight'] = np.log(popularity['sales_value'].values + 1) / sum_log

popularity.drop('sales_value', axis=1, inplace=True)
popularity.head()

CPU times: user 133 ms, sys: 528 µs, total: 133 ms
Wall time: 142 ms


In [10]:
# Проверка
popularity['weight'].sum()

0.9999999999999999

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [12]:
result['weighted_rnd_recommend'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity))
result.head(2)

,user_id,actual,weighted_rnd_recommend
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12731425, 12810248, 6514030, 5567331, 9483956]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1108947, 5589180, 919446, 1039441, 8090440]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [13]:
# your_code
# Отберем топ-5000 популярных товаров
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [14]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [15]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [16]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
K_list = [1, 3, 5, 10, 30]

for k in K_list:
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
    result[f'itemitem_K-{k}'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,  # на вход user-item matrix
                                                                N=5,  # кол-во рекомендаций 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [18]:
result.head(2)

,user_id,actual,weighted_rnd_recommend,itemitem_K-1,itemitem_K-3,itemitem_K-5,itemitem_K-10,itemitem_K-30
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12731425, 12810248, 6514030, 5567331, 9483956]","[6666, 1082185, 995242, 1029743, 840361]","[6666, 1082185, 981760, 995242, 1029743]","[6666, 1082185, 981760, 1127831, 995242]","[6666, 1082185, 981760, 995242, 840361]","[6666, 1082185, 981760, 995242, 1127831]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1108947, 5589180, 919446, 1039441, 8090440]","[6666, 1082185, 1098066, 6534178, 826249]","[6666, 1082185, 981760, 1098066, 6534178]","[6666, 1082185, 981760, 1098066, 995242]","[6666, 1082185, 981760, 1098066, 826249]","[6666, 1082185, 981760, 1098066, 826249]"


In [19]:
# import os, sys

# module_path = os.path.abspath(os.path.join(os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)
    
#from src.metrics import precision_at_k, recall_at_k

In [29]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)
    
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [30]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0014:weighted_rnd_recommend
0.1923:itemitem_K-1
0.1861:itemitem_K-3
0.1368:itemitem_K-5
0.1509:itemitem_K-10
0.1495:itemitem_K-30


Законочерность метрики от количества ближайших соседей, в целом, прослеживается - чем меньше "соседних" user'ов в обучении, тем лучше метрика. За исключением К=5.  
Но на практике, оценивать ItemItemRecommender по 1-3 соседей не всегда правильно.